## Author: Jesús Salazar Gómez, Costa Rica
## Course: Applied Data Science Capstone, week 4 and 5 (project), Coursera

## LinkedIn: https://www.linkedin.com/in/jesagocr/
## GitHub: https://github.com/jesagocr

In [5]:
# to open the url
from urllib.request import urlopen

# to work with the api
import json
import requests
import pandas as pd

# to work with the geolocalization data
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
Libraries imported.


In [3]:
# to open the url
from urllib.request import urlopen

# to work with the api
import json
import requests
import pandas as pd

# to work with the geolocalization data
#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
from bs4 import BeautifulSoup

In [5]:
source = requests.get("https://es.wikipedia.org/wiki/Anexo:Playas_de_Costa_Rica").text
soup = BeautifulSoup(source, 'html.parser')

In [6]:
playas = []

for grossbeach in soup.find_all('li'):

    texto = str(grossbeach.text)

    try:
        if texto[:3] == "Pla" or texto[:3] == "Bal":
            #print('yes')
            playas.append(texto)
        else:
            #no.append(texto)
            pass
    except IndexError:
        pass

In [7]:
def newLine():
    print('********* -*- ********* ')

In [8]:
print(playas)

['Playa Barra del Colorado', 'Playa Bonita', 'Playa Cahuita (Blanca)', 'Playa Chiquita', 'Playa Gandoca', 'Playa Manzanillo', 'Playa Moín', 'Playa Negra (Puerto Viejo)', 'Playa Negra (Cahuita)', 'Playa Pirikiki', 'Playa Portete', 'Playa Puerto Vargas', 'Playas Tortuguero', 'Playa Vizcaya', 'Playa Westfalia', 'Playa Arbolito', 'Playa Avellanas', 'Playa Azul (Jurel)', 'Playa Bahía Junquillal', 'Playa Barco Quebrado', 'Playa Barrigona', 'Playa Bejuco', 'Playa Blanca (Santa Elena)', 'Playa Blanca y Virador', 'Playa Bongo', 'Playa Brasilito', 'Playa Buena Vista', 'Playa Camaronal', 'Playa Carbón', 'Playa Carmen', 'Playa Carrillo', 'Playa Cóncava', 'Playa Conchal', 'Playa Coyote', 'Playa Coyotera', 'Playa Cuajiniquil', 'Playa Culebra', 'Playas del Coco', 'Playa Flamingo', 'Playa Garza', 'Playa Grande', 'Playa Gringo', 'Playa Guiones', 'Playa Hermosa', 'Playa Iguanita', 'Playa Islita y Corozalito', 'Playa Jobo', 'Playa Junquillal', 'Playa La Penca', 'Playa Lagarto', 'Playa Langosta', 'Playa M

In [9]:
print(len(playas))
newLine()

print(playas[-1])
newLine()

playas = playas[:-1]
print(playas[-1])
newLine()

print(len(playas))

137
********* -*- ********* 
Playas de Costa Rica
********* -*- ********* 
Playa Zancudo
********* -*- ********* 
136


In [78]:
url = urlopen("https://es.wikipedia.org/wiki/Anexo:Playas_de_Costa_Rica")

In [82]:
content = str(url.read(),encoding='utf-8')
print(type(content))

<class 'str'>


In [79]:
content = str(url.readline(), encoding='utf-8')
print(type(content))

<class 'str'>


In [83]:
print(content)

mosa</a></li>
<li><a href="/w/index.php?title=Playa_Herradura&amp;action=edit&amp;redlink=1" class="new" title="Playa Herradura (aún no redactado)">Playa Herradura</a></li>
<li><a href="/w/index.php?title=Playa_Isla_Damas&amp;action=edit&amp;redlink=1" class="new" title="Playa Isla Damas (aún no redactado)">Playa Isla Damas</a></li>
<li><a href="/wiki/Jac%C3%B3" title="Jacó">Playa Jacó</a></li>
<li><a href="/w/index.php?title=Playa_Llorona&amp;action=edit&amp;redlink=1" class="new" title="Playa Llorona (aún no redactado)">Playa Llorona</a></li>
<li><a href="/w/index.php?title=Playa_Madrigal&amp;action=edit&amp;redlink=1" class="new" title="Playa Madrigal (aún no redactado)">Playa Madrigal</a></li>
<li><a href="/w/index.php?title=Playa_Mata_de_Lim%C3%B3n&amp;action=edit&amp;redlink=1" class="new" title="Playa Mata de Limón (aún no redactado)">Playa Mata de Limón</a></li>
<li><a href="/w/index.php?title=Playa_Mantas&amp;action=edit&amp;redlink=1" class="new" title="Playa Mantas (aún no r

In [84]:
data = content.split('<li><a href="')
print(data[1])

/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>



In [85]:
for d in data[1:12]:
    print(d)

/wiki/Refugio_Nacional_de_Fauna_Silvestre_Barra_del_Colorado" class="mw-redirect" title="Refugio Nacional de Fauna Silvestre Barra del Colorado">Playa Barra del Colorado</a></li>

/w/index.php?title=Playa_Bonita_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Bonita (Limón) (aún no redactado)">Playa Bonita</a></li>

/w/index.php?title=Playa_Cahuita_(Blanca)&amp;action=edit&amp;redlink=1" class="new" title="Playa Cahuita (Blanca) (aún no redactado)">Playa Cahuita (Blanca)</a></li>

/wiki/Playa_Chiquita" title="Playa Chiquita">Playa Chiquita</a></li>

/w/index.php?title=Punta_Cocles&amp;action=edit&amp;redlink=1" class="new" title="Punta Cocles (aún no redactado)">Punta Cocles</a></li>

/w/index.php?title=Playa_Gandoca&amp;action=edit&amp;redlink=1" class="new" title="Playa Gandoca (aún no redactado)">Playa Gandoca</a></li>

/w/index.php?title=Playa_Manzanillo_(Lim%C3%B3n)&amp;action=edit&amp;redlink=1" class="new" title="Playa Manzanillo (Limón) (aún no redactado)">

In [97]:
playas2 = {}

for d in data[1:-1]:
    inicio = (d.find('">') + 2)
    final = (d.find('</'))
    name = d[inicio:final]

    # it seems the most popular beaches are complete but the others are "aún no redactado"
    ini_red = d.find('aún no')

    if ini_red != -1: # if "aún no redactado" exist is not popular
        playas2[name] = False
    else:
        playas2[name] = True

In [98]:
playas2

{'Playa Barra del Colorado': True,
 'Playa Bonita': False,
 'Playa Cahuita (Blanca)': False,
 'Playa Chiquita': True,
 'Punta Cocles': False,
 'Playa Gandoca': False,
 'Playa Manzanillo': False,
 'Playa Moín': False,
 'Playa Negra (Puerto Viejo)': False,
 'Playa Negra (Cahuita)': True,
 'Playa Pirikiki': False,
 'Playa Portete': False,
 'Playa Puerto Vargas': False,
 'Playas Tortuguero': True,
 'Playa Vizcaya': False,
 'Playa Westfalia': False,
 'Puerto Viejo de Talamanca': True,
 'Punta Mona': True,
 'Punta Uva': False,
 'Playa Arbolito': False,
 'Playa Avellanas': False,
 'Playa Azul (Jurel)': False,
 'Playa Bahía Junquillal': False,
 'Playa Barco Quebrado': False,
 'Playa Barrigona': False,
 'Playa Bejuco': False,
 'Playa Blanca (Santa Elena)': False,
 'Playa Blanca y Virador': False,
 'Playa Bongo': False,
 'Playa Brasilito': False,
 'Playa Buena Vista': False,
 'Playa Camaronal': False,
 'Playa Carbón': False,
 'Playa Carmen': False,
 'Playa Carrillo': True,
 'Playa Cóncava': Fals

In [96]:
len(playas2.keys())

137

In [25]:
print(data)

["b''"]


In [75]:
#to practice let's find the location of the first place in the list

address = 'Playa Zancudo, Costa Rica'

geolocator = Nominatim(user_agent="cr_explorer")
location = geolocator.geocode(address)
print(location)
#latitude = location.latitude
#longitude = location.longitude
#print(type(latitude))
#li.append(latitude)
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

Playa Zancudo, Ferrocarril del Sur, Kilometro 7, Florida, Golfito, Cantón Golfito, 60701, Costa Rica


In [94]:
latitudes = []
longitudes = []

for beach in playas[:7]:
    geolocator = Nominatim(user_agent="cr_explorer_js")
    address = beach+", Costa Rica"
    print(address)
    #print(type(address))

    location = geolocator.geocode(address)

    if location == None:
        latitudes.append(None)
        longitudes.append(None)
        
    else:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)

Playa Barra del Colorado, Costa Rica
Playa Bonita, Costa Rica


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Playa+Bonita%2C+Costa+Rica&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x14F90BB0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

In [95]:
latitudes

[None]

In [96]:
longitudes

[None]